<a href="https://colab.research.google.com/github/Horcruxno13/Multi-Class-Text-Classification-using-BERT-and-PyTorch/blob/main/ECommerce_DistilBERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
ecom_df = pd.read_csv('/content/drive/My Drive/Datasets/ecommerceCleaned.csv')

In [ ]:
ecom_df.head(10)

,Unnamed: 0,Category,Description,Length,Labels
0,0,Household,paper plane design frame wall hang motivationa...,228,3
1,1,Household,saf frame paint wood inch x inch special effec...,59,3
2,2,Household,saf texture modern art print paint synthetic c...,224,3
3,3,Household,saf flower print frame paint synthetic inch x ...,184,3
4,4,Household,incredible gift india wooden happy birthday un...,184,3
5,5,Household,pitaara box romantic venice canvas painting th...,230,3
6,6,Household,paper plane design starry night vangoh wall ar...,275,3
7,7,Household,saf modern art paint synthetic cm x cm x cm se...,213,3
8,8,Household,painting villa uv texture modern art print fra...,111,3
9,9,Household,paint mantra art street jardin bird frame art ...,72,3


In [ ]:
X = ecom_df["Description"].values
y = ecom_df["Labels"].values

In [ ]:
import torch
from torch import nn
from torch.utils.data import TensorDataset, DataLoader
from tqdm import tqdm

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from transformers import DistilBertTokenizer

In [ ]:
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

In [ ]:
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

In [ ]:
input_ids = []
for line in X:
    line = line[:512]
    encoded = tokenizer.encode(line, add_special_tokens = True , padding="longest")
    input_ids.append(encoded)

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
MAX_LENGTH = 512 
input_ids = pad_sequences(input_ids, maxlen=MAX_LENGTH, dtype="long", value=0, truncating="post", padding="post")

In [ ]:
input_ids[:10]

array([[  101,  2526,  4261, ...,     0,     0,     0],
       [  101, 21718,  2087, ...,     0,     0,     0],
       [  101, 21718,  2087, ...,     0,     0,     0],
       ...,
       [  101, 21718,  2087, ...,     0,     0,     0],
       [  101,  3504, 15282, ...,     0,     0,     0],
       [  101,  6628,  1299, ...,     0,     0,     0]])

In [ ]:
attention_masks = []
for line in input_ids:
    mask = [int(token_id > 0) for token_id in line]
    attention_masks.append(mask)

In [ ]:
from sklearn.model_selection import train_test_split 

In [ ]:
input_train, input_remain, label_train, label_remain = train_test_split(input_ids, y, test_size=0.3, random_state=42)
input_valid, input_test, label_valid, label_test = train_test_split(input_remain, label_remain, test_size=0.5, random_state=42)

mask_train, mask_remain, _, _ = train_test_split(attention_masks, y, test_size=0.3, random_state=42)
mask_valid, mask_test, _, _ = train_test_split(mask_remain, label_remain, test_size=0.5, random_state=42)

In [ ]:
import torch
from torch import nn
#from torch.optim import AdamW
from torch.utils.data import TensorDataset, DataLoader
from tqdm import tqdm

In [ ]:
batch_size = 16

trainset = TensorDataset(torch.tensor(input_train), torch.tensor(label_train), torch.tensor(mask_train))
validset = TensorDataset(torch.tensor(input_valid), torch.tensor(label_valid), torch.tensor(mask_valid))
testset = TensorDataset(torch.tensor(input_test), torch.tensor(label_test), torch.tensor(mask_test))

trainloader = DataLoader(trainset, shuffle=True, batch_size=batch_size)
validloader = DataLoader(validset, shuffle=True, batch_size=batch_size)
testloader = DataLoader(testset, shuffle=True, batch_size=batch_size)

In [ ]:
from transformers import DistilBertForSequenceClassification, AdamW

In [ ]:
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels = 4,  output_attentions = False, output_hidden_states = False)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_projector.bias', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.bias', 'classifier.bias', 'classifier.w

In [ ]:
model.cuda()

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0): TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
       

In [ ]:
optimizer = AdamW(model.parameters(), lr = 1e-5, eps = 1e-8)
loss_function = torch.nn.CrossEntropyLoss()

/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
history = {'train_loss': [], 'train_acc': [], 'val_loss': [], 'val_acc': []}

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [ ]:
def flat_accuracy(preds, labels):
  pred_flat = np.argmax(preds, axis=1).flatten()
  labels_flat = labels.flatten()
  return np.sum(pred_flat == labels_flat) / len(labels_flat)
def calcuate_accu(big_idx, targets):
  n_correct = (big_idx==targets).sum().item()
  return n_correct

In [ ]:
epochs = 4
epochloop = tqdm(range(epochs), position=0, desc='Training', leave=True)

for epoch in epochloop:
  model.train()
  train_loss = 0
  train_acc = 0
  n_correct = 0
  nb_tr_steps = 0
  nb_tr_examples = 0
  for id, batch in enumerate(trainloader):
    if id % 50 == 0 and id != 0:
      print("ID Number: ", id)
    inputids = batch[0].to(device)
    inputlabels = batch[1].to(device)
    inputmasks = batch[2].to(device)

    optimizer.zero_grad()
    outputs = model(inputids, attention_mask = inputmasks, return_dict=False)
    loss = loss_function(outputs[0], inputlabels)
    train_loss = train_loss + loss.item()
    big_val, big_idx = torch.max(outputs[0], dim=1)
    n_correct = n_correct + calcuate_accu(big_idx, inputlabels)
    nb_tr_steps += 1
    nb_tr_examples += inputlabels.size(0)
    loss.backward()
    optimizer.step()
  
  print(f'The Total Accuracy for Epoch {epoch}: {(n_correct*100)/nb_tr_examples}')
  epoch_loss = train_loss/nb_tr_steps
  print(f'The Total Loss for Epoch {epoch}: {epoch_loss}')


Training:   0%|          | 0/4 [00:00<?, ?it/s]

ID Number:  50
ID Number:  100
ID Number:  150
ID Number:  200
ID Number:  250
ID Number:  300
ID Number:  350
ID Number:  400
ID Number:  450
ID Number:  500
ID Number:  550
ID Number:  600
ID Number:  650
ID Number:  700
ID Number:  750
ID Number:  800
ID Number:  850
ID Number:  900
ID Number:  950
ID Number:  1000
ID Number:  1050
ID Number:  1100
ID Number:  1150
ID Number:  1200


Training:  25%|██▌       | 1/4 [14:42<44:08, 882.85s/it]

The Total Accuracy for Epoch 0: 37.69076614767997
The Total Loss for Epoch 0: 1.3559561852264719
ID Number:  50
ID Number:  100
ID Number:  150
ID Number:  200
ID Number:  250
ID Number:  300
ID Number:  350
ID Number:  400
ID Number:  450
ID Number:  500
ID Number:  550
ID Number:  600
ID Number:  650
ID Number:  700
ID Number:  750
ID Number:  800
ID Number:  850
ID Number:  900
ID Number:  950
ID Number:  1000
ID Number:  1050
ID Number:  1100
ID Number:  1150
ID Number:  1200


Training:  50%|█████     | 2/4 [29:38<29:40, 890.29s/it]

The Total Accuracy for Epoch 1: 37.96310569857664
The Total Loss for Epoch 1: 1.346276534024868
ID Number:  50
ID Number:  100
ID Number:  150
ID Number:  200
ID Number:  250
ID Number:  300
ID Number:  350
ID Number:  400
ID Number:  450
ID Number:  500
ID Number:  550
ID Number:  600
ID Number:  650
ID Number:  700
ID Number:  750
ID Number:  800
ID Number:  850
ID Number:  900
ID Number:  950
ID Number:  1000
ID Number:  1050
ID Number:  1100
ID Number:  1150
ID Number:  1200


Training:  75%|███████▌  | 3/4 [44:27<14:49, 889.70s/it]

The Total Accuracy for Epoch 2: 37.96310569857664
The Total Loss for Epoch 2: 1.3447667283902134
ID Number:  50
ID Number:  100
ID Number:  150
ID Number:  200
ID Number:  250
ID Number:  300
ID Number:  350
ID Number:  400
ID Number:  450
ID Number:  500
ID Number:  550
ID Number:  600
ID Number:  650
ID Number:  700
ID Number:  750
ID Number:  800
ID Number:  850
ID Number:  900
ID Number:  950
ID Number:  1000
ID Number:  1050
ID Number:  1100
ID Number:  1150
ID Number:  1200


Training: 100%|██████████| 4/4 [59:16<00:00, 889.19s/it]

The Total Accuracy for Epoch 3: 37.96310569857664
The Total Loss for Epoch 3: 1.3449303548455531
